# Product Requirements Document (PRD)

**Project**: LEGO Lean Production Simulation  
**Version**: V0.1  
**Owner**: Yicheng Fan  
**Last Updated**: 2025-11-04  

---

## 1. Project Background

To enhance understanding and practical application of Lean Production principles, this simulation system is built using LEGO as a physical abstraction. Although not based on a real-world factory floor, by decomposing the production flow, assigning teams, and simulating material and information flow, we aim to replicate key variables and types of waste (e.g., waiting, transportation, inventory) in manufacturing and use the simulation for performance assessment and continuous improvement.

---

## 2. Objectives

- Build an abstract but functional model of a lean production system  
- Implement a simulation engine using Python (or R) with interactive execution  
- Allow configurable inputs and random events to test various lean strategies  
- Support iterative experiments for comparison and optimization  

---

## 3. Conceptual Modeling

### 3.1 Team Assignments

- Team 1: Type Sorting  
- Team 2: Set Sorting  
- Team 3: Axis Assembly  
- Team 4: Chassis Assembly  
- Team 5: Final Assembly  

### 3.2 Resource Objects

- Raw materials (Bricks)  
- Intermediate warehouses (B, C1, C2, C3, D1, D2)  
- Finished goods warehouse (E)  
- Tasks/Orders (Gliders)  

### 3.3 Model Elements

- **State Variables**: inventory levels, queue lengths, completed items, simulation clock  
- **Parameters**: average process time, team size, transport time, shift length, initial stock  
- **Random Events**: missing bricks, rework, transport delay, human error  
- **Assumptions**: FIFO processing, fixed initial inventory, one task per worker, one task per team per round  

---

## 4. Core Functional Requirements

### 4.1 Simulation Engine

- Event-driven or time-stepped processing  
- Each team action modeled as a processing node  
- Support for both step-by-step and auto-run modes  

### 4.2 Parameter Configuration

- User-defined process times (constant or distribution-based)  
- Team size, buffer size, initial inventory, task generation pace  

### 4.3 Visualization & Interaction

- Show real-time buffer status, queues per node  
- Live KPI dashboard  
- Pause, reset, and playback capabilities  

### 4.4 Random Event Simulation

- Probability-based disruptions (e.g., missing parts, rework, blocking)  
- Log all exceptions and reflect impact on KPIs  

---

## 5. Non-Functional Requirements

- Response time ≤ 1s per step  
- Local single-user deployment (CLI or simple Web UI)  
- Modular model structure for future expansion  
- Use `pandas` for data tracking, `matplotlib`/`plotly` for visualization  

---

## 6. KPI Design and Tracking

| KPI Name        | Description                          | Calculation Method                             |
|-----------------|--------------------------------------|-------------------------------------------------|
| Throughput      | Completed gliders per unit time      | Total completed / simulation time              |
| Lead Time       | Avg time per glider from start to end| Completion time - start time (average)         |
| WIP Level       | Items being processed                | Time-weighted average count of in-process items|
| Inventory Level | Items in buffers                     | Periodic snapshot of each warehouse stock      |
| Utilization     | Resource usage rate                  | Busy time / total time per team                |
| Error Rate      | Rate of disruptions                  | Exception count / total operations             |

---

## 7. Data Interface (Initial)

| Name          | Type   | Example | Description                            |
|---------------|--------|---------|----------------------------------------|
| process_time  | float  | 2.5     | Avg process time per task (sec)        |
| team_size     | int    | 3       | Number of people per team              |
| buffer_limit  | int    | 10      | Max warehouse capacity                 |
| shift_length  | int    | 480     | Shift length in minutes                |
| error_rate    | float  | 0.05    | Probability of a random disruption     |

---

## 8. Next Steps

1. Await detailed configuration: team setup, process times, error probabilities  
2. Build initial Python discrete-event simulation script (with logging and basic visualization)  
3. Deliver first working prototype in 2 weeks with minimum closed-loop simulation and KPI output (Front-end needed)  
